In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import os
import time
from src import (zona_prop_url)

In [ ]:
options = Options()
options.add_argument("--headless") # Ensures GUI is off
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

class Scraper:
    def __init__(self):
        self.driver = None

    def inicio_driver(self, link: str):
        if self.driver is None:
            service = Service(ChromeDriverManager().install())
            carpeta_descarga = os.getcwd() + "/data"
            prefs = {
                'download.default_directory': carpeta_descarga,
                "directory_upgrade": True
            }
            chrome_options = webdriver.ChromeOptions()
            chrome_options.add_experimental_option("prefs", prefs)
            self.driver = webdriver.Chrome(service=service, options=chrome_options)
            self.driver.maximize_window()
        self.driver.get(link)
        return self.driver

scraper = Scraper()
driver = scraper.inicio_driver(zona_prop_url)

html = driver.page_source
# driver.quit()

soup = BeautifulSoup(html, 'html.parser')

# Select all the div elements with the required class
property_elements = soup.select('div.sc-1tt2vbg-4.dFNvko')

# Initialize the lists to store the data
prices = []
locations = []
addresses = []
features = []
descriptions = []
expensas = []

# Iterate over each property element
for property_element in property_elements:
    # Find the child elements with the required data-qa attributes
    price_element = property_element.select_one('div[data-qa="POSTING_CARD_PRICE"]')
    location_element = property_element.select_one('div[data-qa="POSTING_CARD_LOCATION"]')
    address_element = property_element.select_one('div[class="sc-ge2uzh-0 eXwAuU"]')
    features_elements = property_element.select('div[data-qa="POSTING_CARD_FEATURES"] span')
    description_element = property_element.select_one('div[data-qa="POSTING_CARD_DESCRIPTION"]')
    expensas_element = property_element.select_one('div[data-qa="expensas"]')

    # Extract the text from the elements and append it to the lists
    prices.append(price_element.text if price_element else np.nan)
    locations.append(location_element.text if location_element else np.nan)
    addresses.append(address_element.text if address_element else np.nan)
    features.append([feature_element.text for feature_element in features_elements])
    descriptions.append(description_element.text if description_element else np.nan)
    expensas.append(expensas_element.text if expensas_element else np.nan)
    

# Create a DataFrame from the lists
df = pd.DataFrame({
    'Price': prices,
    'Location': locations,
    'Adress': addresses,
    'Features': features,
    'Description': descriptions,
    'Expensas': expensas
})

df

In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import os
import time

options = Options()
options.add_argument("--headless") # Ensures GUI is off
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

class Scraper:
    def __init__(self):
        self.driver = None

    def inicio_driver(self, link: str):
        if self.driver is None:
            service = Service(ChromeDriverManager().install())
            carpeta_descarga = os.getcwd() + "/data"
            prefs = {
                'download.default_directory': carpeta_descarga,
                "directory_upgrade": True
            }
            chrome_options = webdriver.ChromeOptions()
            chrome_options.add_experimental_option("prefs", prefs)
            self.driver = webdriver.Chrome(service=service, options=chrome_options)
            self.driver.maximize_window()
        self.driver.get(link)
        return self.driver

scraper = Scraper()
driver = scraper.inicio_driver(zona_prop_url)

# Initialize the lists to store the data
prices = []
locations = []
addresses = []
features = []
descriptions = []
expensas = []

# Start scraping from the first page
while True:
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # Select all the div elements with the required class
    property_elements = soup.select('div.sc-1tt2vbg-4.dFNvko')

    # Iterate over each property element
    for property_element in property_elements:
        # Find the child elements with the required data-qa attributes
        price_element = property_element.select_one('div[data-qa="POSTING_CARD_PRICE"]')
        location_element = property_element.select_one('div[data-qa="POSTING_CARD_LOCATION"]')
        address_element = property_element.select_one('div[class="sc-ge2uzh-0 eXwAuU"]')
        features_elements = property_element.select('div[data-qa="POSTING_CARD_FEATURES"] span')
        description_element = property_element.select_one('div[data-qa="POSTING_CARD_DESCRIPTION"]')
        expensas_element = property_element.select_one('div[data-qa="expensas"]')

        # Extract the text from the elements and append it to the lists
        prices.append(price_element.text if price_element else np.nan)
        locations.append(location_element.text if location_element else np.nan)
        addresses.append(address_element.text if address_element else np.nan)
        features.append([feature_element.text for feature_element in features_elements])
        descriptions.append(description_element.text if description_element else np.nan)
        expensas.append(expensas_element.text if expensas_element else np.nan)

    # Check if there is a next page
    next_page = soup.select_one('a[data-qa="PAGING_NEXT"]')
    if next_page and next_page['href']:
        # Prepend the base URL to the href
        full_url = zona_prop_url + next_page['href']
        # Go to the next page
        driver.get(full_url)
    else:
        # No more pages, break the loop
        break

# Create a DataFrame from the lists
df = pd.DataFrame({
    'Price': prices,
    'Location': locations,
    'Address': addresses,
    'Features': features,
    'Description': descriptions,
    'Expensas': expensas
})

df


,Price,Location,Address,Features,Description,Expensas
0,USD 21.895,"Retiro, Capital Federal",Arenales 707,"[ 755 m² , 755 m² , 755 m² , 755 m² , 7 co...","Características: • Edificio plaza san martin, ...",NaN
1,USD 1.000,"Villa Urquiza, Capital Federal",Monroe al 5600,"[ 90 m² , 90 m² , 90 m² , 90 m² , 2 baños ...",Excelente Local comercial en alquiler!! Ubicad...,NaN
2,$ 1.100.000,"Belgrano C, Belgrano",Libertador al 6200,"[ 75 m² , 75 m² , 70 m² , 70 m² , 1 baño ,...",Hermosa oficina con muy buena luz natural por ...,$ 80.000 Expensas
3,USD 700,"Belgrano, Capital Federal",Monroe al 2400,"[ 36 m² , 36 m² , 1 amb. , 1 amb. , 1 baño...",No apto alquiler temporariomonoambiente premiu...,$ 30.000 Expensas
4,USD 4.700,"Belgrano, Capital Federal",Avenida del Libertador al 6500,"[ 233 m² , 233 m² , 233 m² , 233 m² , 2 ba...","Edificio desarrollado en Subsuelo, Sótano, Ent...",NaN
5,USD 11.500,"Centro / Microcentro, Capital Federal",Corrientes 1454,"[ 525 m² , 525 m² , 525 m² , 525 m² ]","A metros del Obelisco, sobre la remozada Av. C...",NaN
6,USD 12.000,"Barrio Norte, Capital Federal",DEL Libertador AV. 4444,"[ 400 m² , 400 m² , 400 m² , 400 m² , 5 am...",-descripción: -excelente piso de categoria tot...,$ 60.000 Expensas
7,USD 6.000,"Palermo Chico, Palermo",Ortiz de Ocampo al 3300,"[ 105 m² , 105 m² , 105 m² , 105 m² , 5 am...",Inmejorable oficina practicamente a estrenar e...,$ 318.613 Expensas
8,USD 2.300,"Las Cañitas, Palermo",Migueletes 1231,"[ 183 m² , 183 m² , 173 m² , 173 m² , 2 co...","Edificio ""ambiant bureaux"". Hall de categoría ...",NaN
9,$ 400.000,"Colegiales, Capital Federal",Teodoro García 2593 - Caba,"[ 53 m² , 53 m² , 50 m² , 50 m² , 2 amb. ,...",2 Ambientes en dúplex. Inmejorable ubicación (...,$ 60.000 Expensas
